## Compare ice water path and effective radii

In [1]:
import os
import netCDF4 as nc
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

Define paths where to find the data. Cloudnet data must be downloaded from Pangaea first!

In [2]:
fname_tcwret = os.getenv('HOME') + '/code_richter_et_al/data_TCWret/mixed_ice_shape.nc'
path_cnet = os.getenv('HOME') + '/code_richter_et_al/get_cloudnet'

Read data from TCWret and save them as Pandas.DataFrame

In [3]:
with nc.Dataset(fname_tcwret) as f:
    #print(f.variables)
    seconds = f.variables['time_of_measurement'][:]
    iwp = f.variables['ice_water_path'][:]
    iwp_err = f.variables['ice_water_path_error'][:]
    rliq = f.variables['liquid_water_effective_droplet_radius'][:]
    rice = f.variables['ice_water_effective_droplet_radius'][:]
    red_chi_2 = f.variables['reduced_chi_2'][:]
    t_cw = f.variables['liquid_water_optical_depth'][:] + f.variables['ice_water_optical_depth'][:]
    dof = f.variables['degrees_of_freedom_of_signal'][:]
    pwv = f.variables['precipitable_water_vapour'][:]
    type_ice = f.variables['ice_shape'][:]
            
time = np.array([])
for ii in range(len(seconds)):
    sec = int(seconds[ii])
    time = np.concatenate((time, [dt.timedelta(seconds=sec) + dt.datetime(2017, 5, 1)]))
    
tcwret_raw = pd.DataFrame({'time': time, 'pwv(cm)': pwv, 'rliq(um)': rliq, 'rice(um)': rice, 'iwp(gm-2)': iwp, 'diwp(gm-2)': iwp_err, 'red_chi_2(1)': red_chi_2, 'tcw(1)': t_cw, 'dof(1)': dof})

Apply filtering of data

In [4]:
tau_max = 6.0
tau_min = 0.0

idx_conv = np.where((tcwret_raw['red_chi_2(1)'] <= 1.0) & (tcwret_raw['red_chi_2(1)'] >= 0.0))[0]
idx_tau = np.where((tcwret_raw['tcw(1)'] <= tau_max) & (tcwret_raw['tcw(1)'] >= tau_min))[0]
idx_valid = np.intersect1d(idx_conv, idx_tau)
tcwret = tcwret_raw.iloc[idx_valid]

counter = 0
idx = np.array([])
for ii in range(len(tcwret)):
    if tcwret['rliq(um)'].iloc[ii] < tcwret['rice(um)'].iloc[ii]:
        idx = np.concatenate((idx, [ii]))
        counter += 1
tcwret = tcwret.iloc[idx]

Read Cloudnet data. Only allow retrieval flags 0,1,3 and 4

In [5]:
iwc_st_invalid = [2,5,6]#,7]
cloudnet = {'time': [], 'iwp(gm-2)': [], 'rice(um)': [], 'diwp(gm-2)': []}
for file_ in sorted(os.listdir(path_cnet)):
    if ".nc" in file_:
        with nc.Dataset(os.path.join(path_cnet, file_)) as f:
            if "cnet" not in file_:
                continue
            day_month = dt.datetime.strptime(file_, 'cnet_%m_%d.nc')
            day = dt.datetime(2017, day_month.month, day_month.day)
            #latitude = f.variables['latitude'][:]
            #longitude = f.variables['longitude'][:]
            time = f.variables['datetime'][:]
            height = f.variables['height'][:]
            lwp = f.variables['liquid_water_path_per_layer'][:]
            lwp_st = f.variables['liquid_water_content_status'][:]
            iwc = f.variables['ice_water_content'][:]
            iwp = f.variables['ice_water_path_per_layer'][:]
            iwp_st = f.variables['ice_water_content_status'][:]
            iwp_err = f.variables['ice_water_content_error'][:]
            lwp_mwr = f.variables['liquid_water_path_MWR'][:]
            lwp_err_mwr = f.variables['liquid_water_path_error_MWR'][:]
            rliq = f.variables['reff_Frisch'][:]
            rice = f.variables['reff_ice'][:]
            rliq_st = f.variables['reff_Frisch_status'][:]
            dz = np.diff(height)
            dz = np.concatenate((dz, [0]))
            for time_idx in range(len(time)):
                time_iter = day + dt.timedelta(seconds=int(np.round(time[time_idx]*3600)))
                idx_liq = np.where(lwp[time_idx] > 0.0)[0]
                idx_ice = np.where(iwp[time_idx] > 0.0)[0]

                iwc_invalid = np.intersect1d(iwp_st[time_idx], np.array(iwc_st_invalid))
                if iwc_invalid.size != 0: continue
                if idx_ice.size == 0:
                    rice_sum = 0.0 
                else:
                    rice_sum = np.mean(rice[time_idx, idx_ice])
                    iwc_err_abs = iwc[time_idx, idx_ice]*10**iwp_err[time_idx, idx_ice]*1e-2#1e-2 wegen Prozent

                    diwp = iwc_err_abs * dz[idx_ice]
                    diwp_sum = np.sum(diwp)
                    #diwp_sum = np.sum(iwp_err[time_idx, idx_ice]*dz[idx_ice])
                cloudnet['rice(um)'].append(rice_sum)
                cloudnet['time'].append(time_iter)
                cloudnet['iwp(gm-2)'].append(np.sum(iwp[time_idx]))
                cloudnet['diwp(gm-2)'].append(diwp_sum)
cloudnet = pd.DataFrame(cloudnet)
idx = np.array([])
for ii in range(len(cloudnet)):
    if  not np.ma.is_masked(cloudnet['iwp(gm-2)'].iloc[ii]) and not np.ma.is_masked(cloudnet['diwp(gm-2)'].iloc[ii]) and not np.ma.is_masked(cloudnet['rice(um)'].iloc[ii]):
        idx = np.concatenate((idx, [ii]))
idx = np.array(idx, dtype=int)
cloudnet = cloudnet.iloc[idx]

Define averaging time interval

In [6]:
delta = 2

Average Cloudnet data

In [7]:
lwp_mean = []
lwp_mean_mwr = []
iwp_mean = []
diwp_mean = []
cwp_mean = []
rliq_mean = []
rice_mean = []
lat_mean = []
lon_mean = []
time_mean = []
lwp_mean_err_mwr = []
datetime_start = np.datetime64("2017-05-24T20:25:00")
datetime_iter = datetime_start
datetime_stop = np.datetime64("2017-07-18T00:00:00")
while datetime_iter < datetime_stop:
    idx = np.where((np.array(cloudnet['time']) > datetime_iter) & \
                   (np.array(cloudnet['time']) < datetime_iter+np.timedelta64(delta*60, 's')))[0]
    if idx.size != 0:
        iwp_mean.append(np.mean(np.array(cloudnet['iwp(gm-2)'])[idx]))
        diwp_mean.append(np.mean(np.array(cloudnet['diwp(gm-2)'])[idx]))
        rice_mean.append(np.mean(np.array(cloudnet['rice(um)'])[idx]))
        time_mean.append(datetime_iter)
    datetime_iter += np.timedelta64(delta*60, 's')
    
cloudnet_av = pd.DataFrame({'time': time_mean, 'iwp(gm-2)': iwp_mean, 'diwp(gm-2)': diwp_mean, 'rice(um)': rice_mean})

TypeError: invalid type promotion

Average TCWret data

In [ ]:
lwp_mean = []
iwp_mean = []
diwp_mean = []
cwp_mean = []
rliq_mean = []
rice_mean = []
lat_mean = []
lon_mean = []
time_mean = []
pwv_mean = []
chi_mean = []
dof_mean = []
datetime_start = np.datetime64("2017-05-24T20:25:00")
datetime_iter = datetime_start
datetime_stop = np.datetime64("2017-07-18T00:00:00")
while datetime_iter < datetime_stop:
    idx = np.where((np.array(tcwret['time']) > datetime_iter) & \
                   (np.array(tcwret['time']) < datetime_iter+np.timedelta64(delta*60, 's')))[0]
    if idx.size != 0:
        iwp_mean.append(np.mean(np.array(tcwret['iwp(gm-2)'])[idx]))
        diwp_mean.append(np.mean(np.array(tcwret['diwp(gm-2)'])[idx]))
        rice_mean.append(np.mean(np.array(tcwret['rice(um)'])[idx]))
        pwv_mean.append(np.mean(np.array(tcwret['pwv(cm)'])[idx]))
        chi_mean.append(np.mean(np.array(tcwret['red_chi_2(1)'])[idx]))
        dof_mean.append(np.mean(np.array(tcwret['dof(1)'])[idx]))
        time_mean.append(datetime_iter)
    datetime_iter += np.timedelta64(delta*60, 's')
    
tcwret_av = pd.DataFrame({'time': time_mean, \
                          'iwp(gm-2)': iwp_mean, \
                          'rice(um)': rice_mean, \
                          'diwp(gm-2)': diwp_mean, \
                          'pwv(cm)': pwv_mean, \
                          'red_chi_2(1)': chi_mean, \
                          'dof(1)': dof_mean})

Calculate correlation coefficient, p-Value, mean and standard deviation

In [ ]:
intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
xax = np.array(tcwret_av['iwp(gm-2)'].iloc[idx_tcwret])
yax = np.array(cloudnet_av['iwp(gm-2)'].iloc[idx_cloudnet])
pearsonr, pval = scipy.stats.pearsonr(xax,yax)

print("Data\t\t\tcor\tp-Value\tMean\tSD\tNumber")
print("IWP All\t\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax-yax), np.std(xax-yax), xax.size))

intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
cax = np.array(tcwret_av['pwv(cm)'].iloc[idx_tcwret])
idx = np.where(cax < 1.0)[0]
xax = xax[idx]
yax = yax[idx]
#xax = np.array(tcwret_av['iwp(gm-2)'].iloc[idx_tcwret])
#yax = np.array(cloudnet_av['iwp(gm-2)'].iloc[idx_cloudnet])
pearsonr, pval = scipy.stats.pearsonr(xax,yax)

print("IWP PWV < 1.0\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax-yax), np.std(xax-yax), xax.size))

intersect, idx_tcwret, idx_cloudnet = np.intersect1d(tcwret_av['time'], cloudnet_av['time'], return_indices=True)
idx_rice = np.where((xax > 0.0) & (yax > 0.0))[0]
xax_rice = np.array(tcwret_av['rice(um)'].iloc[idx_tcwret])[idx_rice]
yax_rice = np.array(cloudnet_av['rice(um)'].iloc[idx_cloudnet])[idx_rice]
pearsonr, pval = scipy.stats.pearsonr(xax_rice,yax_rice)

print("rice\t\t\t{:.2f}\t{:.2f}\t{:.2f}\t{:.2f}\t{}".format(pearsonr, pval, np.mean(xax_rice-yax_rice), np.std(xax_rice-yax_rice), xax_rice.size))